In [26]:
import os
from dotenv import load_dotenv

# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()
# Acessando a variável de ambiente API_KEY
os.environ['OPENAI_API_KEY'] = os.getenv("MY_KEY")

## LLMs

In [27]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)



"Maharaja's Palace: A Taste of India"


In [28]:
llm("I want to open a restaurant for Indian food. Suggest a fency name for this.")

c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"Maharaja\'s Spice Palace"'

## Prompt Templates

In [29]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)


I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [30]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\n"La Cantina de México"'

In [31]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fency name for this.

> Finished chain.


'\n\n"Rancho de Sabor"'

In [32]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [33]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Indian")
print(content)



1. Tandoori Chicken Skewers
2. Vegetable Samosas
3. Butter Chicken
4. Lamb Vindaloo
5. Chana Masala
6. Palak Paneer
7. Garlic Naan
8. Vegetable Biryani
9. Mango Lassi
10. Masala Chai Tea 



#### Sequential Chain

In [34]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [35]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [36]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [37]:
chain({"cuisine": "Indian"})

c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Maharaja\'s Feast" ',
 'menu_items': '\n\n1. Tandoori Chicken: Traditional Indian dish of marinated chicken cooked in a clay oven.\n\n2. Butter Chicken: Slow-cooked chicken in a rich and creamy tomato-based gravy.\n\n3. Lamb Biryani: Fragrant basmati rice cooked with tender chunks of lamb, spices, and herbs.\n\n4. Palak Paneer: Spinach and cheese dish cooked in a creamy onion and tomato-based gravy.\n\n5. Naan Bread: Soft and fluffy Indian flatbread, perfect for scooping up curries and sauces.\n\n6. Samosas: Crispy fried pastry filled with spiced potatoes and peas.\n\n7. Dal Makhani: Creamy lentil dish made with black lentils, kidney beans, and spices.\n\n8. Vegetable Korma: Mixed vegetables cooked in a creamy and mildly spiced cashew-based sauce.\n\n9. Mango Lassi: Refreshing yogurt-based drink with mango puree and a hint of cardamom.\n\n10. Gulab Jamun: Deep-fried dough balls soaked in a sweet syrup, a popular Indian dessert.'}

## Agents

In [39]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11706 sha256=c13994d0dbfcbd4dac3f110773ae4314666ba5d0d01c30caaa2c520c80edeae3
  Stored in directory: c:\users\luan lima\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
# install this package: pip install wikipedia

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

# Let's test it out!
agent.run("When was Elon musk born? What is his age right now in 2023?")

c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about Elon Musk's birthdate and current age.
Action: Wikipedia
Action Input: Elon Musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and investor. He is the founder, chairman, CEO, and CTO of SpaceX; angel investor, CEO, product architect, and former chairman of Tesla, Inc.; owner, executive chairman, and CTO of X Corp.; founder of the Boring Company and xAI; co-founder of Neuralink and OpenAI; and president of the Musk Foundation. He is one of the wealthiest people in the world; as of April 2024, Forbes estimates his net worth to be $178 billion. 
A member of the wealthy South African Musk family, Musk was born in Pretoria and briefly attended the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada. Musk la

'Elon Musk will be 52 years old in 2023.'

## Memory

In [41]:
chain = LLMChain(llm=llm,prompt=prompt_template_name)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [42]:
name = chain.run("Indian")
print(name)



"Spice Palace"


In [43]:
chain.memory

In [44]:
type(chain.memory)

NoneType

#### ConversationBufferMemory

In [45]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabor Exquisito" (The Exquisite Flavor)


In [46]:
name = chain.run("Arabic")
print(name)



"Al-Fawarish" (meaning "The Treasures" in Arabic)


In [47]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabor Exquisito" (The Exquisite Flavor)
Human: Arabic
AI: 

"Al-Fawarish" (meaning "The Treasures" in Arabic)


#### ConversationChain

In [48]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [49]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the final match by 17 runs. The tournament was held in England and was organized by the International Cricket Council. The West Indies team was captained by Clive Lloyd and they had an impressive lineup of players, including Viv Richards, Gordon Greenidge, and Michael Holding. It was a historic moment for cricket and remains one of the most iconic world cup victories to date.'

In [50]:
convo.run("How much is 5+5?")

' 5+5 is equal to 10. This is a basic mathematical equation and the answer is derived from the fact that 5+5 is the same as 10. This is known as addition in mathematics, which is the process of combining two or more numbers to get a total sum. In this case, we are adding two numbers, 5 and 5, to get the total of 10. This is a simple concept but can be used to solve more complex mathematical problems.'

In [51]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the first cricket world cup in 1975 was Clive Lloyd. He was a prominent cricketer from the West Indies and led his team to victory in the final match against Australia. Lloyd was known for his strong leadership skills and was also a key player in the team, scoring 102 runs in the final match. He went on to captain the West Indies team for many years and is considered one of the greatest captains in cricket history.'

In [52]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies team in 1975. They beat Australia in the final match by 17 runs. The tournament was held in England and was organized by the International Cricket Council. The West Indies team was captained by Clive Lloyd and they had an impressive lineup of players, including Viv Richards, Gordon Greenidge, and Michael Holding. It was a historic moment for cricket and remains one of the most iconic world cup victories to date.
Human: How much is 5+5?
AI:  5+5 is equal to 10. This is a basic mathematical equation and the answer is derived from the fact that 5+5 is the same as 10. This is known as addition in mathematics, which is the process of combining two or more numbers to get a total sum. In this case, we are adding two numbers, 5 and 5, to get the total of 10. This is a simple concept but can be used to solve more complex mathematical problems.
Human: Who was the captain ofthe winning team?
AI

#### ConversationBufferWindowMemory

In [53]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was held in 1975 and was won by the West Indies team. They defeated Australia in the final match by 17 runs. This was a historic moment for the West Indies as it was their first time winning the world cup.'

In [54]:
convo.run("How much is 5+5?")

" The answer to 5+5 is 10. Did you know that the concept of addition dates back to ancient civilizations, such as the Egyptians and Babylonians? They used a form of addition to keep track of their resources and trade. It wasn't until the 12th century that the Hindu-Arabic numeral system, which is the basis for modern addition, was developed."

In [55]:
convo.run("Who was the captain of the winning team?")

' I do not have enough information to answer that question. Do you know which team you are referring to?'